<a href="https://colab.research.google.com/github/Yewon-dev/boostcamp-AI-Tech/blob/main/AI-Paper-Review/ResNet_paper_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ResNet
*Deep Residual Learning for Image Recognition*



-----




## Abstract
Plain networks는 layer가 깊어질수록 성능이 향상될 것이라는 예상과 다르게 vanishing/exploding gradients 문제가 발생한다. 따라서 본 논문에서는 이런 degradation 문제를 해결하기 위해서 deep residual learning framework를 제안한다.



## Introduction


### Shortcut(Skip) Connection
<img src="https://drive.google.com/uc?export=view&id=1JDAcjcChC770k8EifDB01-xluOxfxzZr" width=400>

기존 출력인 $H(x)$를 직접 학습하기 보다는, 더 학습하기 쉬운 $F(x) := H(x)-x$를 대신 학습시킨다. 결과적으로 Shortcut connection은 $F(x)$에 $x$를 더해주는 것을 의미하고, 이를 단순히 identity mapping으로 사용할 수 있다. 추가적인 parameter나 복잡도가 증가하지 않는 것과 구현이 간단하다는 것이 장점이다.

또한 ($F(x)$가 0이 되도록 학습하여) $H(x) = 0 + x$를 미분했을 때 x가 1이 되기 때문에 vanishing gradient 문제가 해결된다.

vanishing gradient 문제가 해결되면 accuracy가 감소하지 않으면서 layer를 더 깊게 쌓을 수 있다.

-----

## Architecture
아래의 그림에서 34-layer residual에 shortcut connection이 추가된 것을 볼 수 있다.
convolution filter를 두개씩 묶어서 residual function 형태로 학습을 시켰고,
점선으로 표시된 부분은 input/output dimension이 달라서 dimension을 matching 시키는 함수를 적용했다.





<img src="https://drive.google.com/uc?export=view&id=1hDQ29_DqojvD1m692Yoekb_W-D-0Mgoq" width=400>

하나의 building block을 다음과 같이 정의한다.
$$y = F(x,\{W_i\})+x$$
$F$는 residual mapping을 의미하고, $x$는 identity mapping(=shortcut connection)을 의미한다.

(Figure 2.는 $F_1=W_2σ(W_1x)$라고 할 수 있다.)
  


- input과 output의 dimension이 동일할 때, identity mapping 수행한다.
-  그렇지 않다면 세가지 옵션 사용한다.
  1. (A) zero padding을 증가하는 차원에 적용하여, identity mapping 수행한다.
  2. (B) 차원이 증가할 때만 projection 연산을 활용한 shortcut connection 구현한다.
> $x$에 linear projection $W_s$를 곱해줌으로써 dimension을 일치시킨다.
$$y = F(x,\{W_i\})+W_sx$$
  3. (C) 모든 shortcut에 대해 projection을 사용한다.

<img src="https://drive.google.com/uc?export=view&id=1lbP_-K-Tn2NAbXkE1XPc_7-JqHCcdCKA" width=400>

Table 3.를 보면 (C)가 error가 가장 낮은 것을 볼 수 있지만 본 논문에서는 복잡도와 모델 사이즈를 줄이기 위해 (C)는 사용하지 않을 것이다. identity mapping으로도 성능을 많이 개선시킬 수 있고, bottleneck 구조에서 복잡도를 증가시키지 않기 위해서 identity mapping를 효과적으로 사용할 수 있다.

-----


### Bottleneck 

<img src="https://drive.google.com/uc?export=view&id=1tWXTKSePHww_rvBU7QRNz3VaQxNoMPNu" width=400>

신경망의 시작과 끝에 1x1 conv layers를 추가함으로써 성능을 감소시키지 않고 parameter의 수를 감소시킬 수 있다. 따라서 더 깊은 신경망을 적은 복잡도로 구현할 수 있다.

<img src="https://drive.google.com/uc?export=view&id=1kfUajPVQDrekdTni4Wz7kkPTl-02mOoS" width=600>

ResNet-152가 VGG-16보다 더 적은 연산량을 갖고 있다. *(VGG-16 : 19.6 billion FLOPs)*

# Implementing ResNet-18 in Pytorch

- 출처 https://github.com/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/ResNet18_CIFAR10_Train.ipynb


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import os


# ResNet18을 위해 최대한 간단히 수정한 BasicBlock 클래스 정의
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()

        # 3x3 필터를 사용 (너비와 높이를 줄일 때는 stride 값 조절)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        # 3x3 필터를 사용 (패딩을 1만큼 주기 때문에 너비와 높이가 동일)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        self.shortcut = nn.Sequential() # identity인 경우
        if stride != 1: # stride가 1이 아니라면, Identity mapping이 아닌 경우
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x) # (핵심) skip connection
        out = F.relu(out)
        return out


# ResNet 클래스 정의
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        # 64개의 3x3 필터(filter)를 사용
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        # Fully Connected layer
        self.linear = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes # 다음 레이어를 위해 채널 수 변경
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


# ResNet18 함수 정의
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])